In [1]:
import pandas as pd 
from optbinning import OptimalBinning
from sklearn.impute import SimpleImputer
from sklearn.tree import DecisionTreeClassifier

In [2]:
df = pd.read_csv('gen_data.csv')

### Tính đầy đủ 

In [3]:
pd.set_option('display.max_rows', None)

Tỷ lệ missing theo cột

In [4]:
missing_col = pd.DataFrame({
    'missing_count': df.isna().sum(),
    'missing_pct': df.isna().mean() * 100
}).sort_values('missing_pct', ascending=False)

missing_col

,missing_count,missing_pct
SOCIF,0,0.0
HAS_MIDTERM_LOAN,0,0.0
CBAL_LONGTERM_LOAN,0,0.0
CBAL_MIDTERM_LOAN,0,0.0
CBAL_SHORTTERM_LOAN,0,0.0
IIP_GROWTH_12M,0,0.0
IIP,0,0.0
UR_GROWTH_12M,0,0.0
UR,0,0.0
CPI_GROWTH_12M,0,0.0


In [5]:
pd.set_option('display.max_rows', 20)

Tỷ lệ missing theo row

In [6]:
df['ROW_MISSING_PCT'] = df.isna().mean(axis=1) * 100

df['ROW_MISSING_PCT'].describe()


count    2400000.0
mean           0.0
std            0.0
min            0.0
25%            0.0
50%            0.0
75%            0.0
max            0.0
Name: ROW_MISSING_PCT, dtype: float64

In [7]:
df['FLAG_HIGH_MISSING'] = (df['ROW_MISSING_PCT'] > 40).astype(int)

### Tính duy nhất 

Trùng khóa logic (SOCIF – year)

In [8]:
dup_key = df.duplicated(subset=['SOCIF', 'year'], keep=False)

df.loc[dup_key].shape

(0, 77)

Trùng toàn bộ bản ghi (record duplicate)

In [9]:
dup_full = df.duplicated(keep=False)

dup_full.sum()


np.int64(0)

In [10]:
dup_rate = dup_full.mean() * 100
print(f"Tỷ lệ bản ghi trùng lặp: {dup_rate:.2f}%")


Tỷ lệ bản ghi trùng lặp: 0.00%


### Tính kịp thời 

Check dùng future information (leakage)

In [11]:
# BAD_NEXT_12M chỉ được missing ở năm cuối
leak_check = df[
    (df['year'] < df['year'].max()) &
    (df['BAD_NEXT_12M'].isna())
]

leak_check.shape


(0, 77)

Check continuity theo năm

In [12]:
year_count = df.groupby('year')['SOCIF'].nunique()
year_count


year
2018    400000
2019    400000
2020    400000
2021    400000
2022    400000
2023    400000
Name: SOCIF, dtype: int64

### Tính phù hợp 

In [13]:
rule_valid_age = (
    df['TUOI'].between(0, 120)
)

df.loc[~rule_valid_age].shape


(160, 77)

In [14]:
rule_valid_ltv = df['LTV'].between(0, 400)

df.loc[~rule_valid_ltv].shape


(30, 77)

In [15]:
rule_valid_duration = (
    (df['DURATION_MAX'] >= 0) &
    (df['REMAINING_DURATION_MAX'] >= 0)
)

df.loc[~rule_valid_duration].shape


(20, 77)

In [16]:
valid_gender = df['C_GIOITINH'].isin(['M', 'F', 'O'])
df.loc[~valid_gender].shape


(0, 77)

In [17]:
df['FLAG_INVALID'] = ~(
    rule_valid_age &
    rule_valid_ltv &
    rule_valid_duration &
    valid_gender
)


In [18]:
pd.set_option('display.max_columns', None)

In [19]:
df.describe()

,SOCIF,BASE_AGE,TRINHDO,SOHUUNHA,NHANVIENBIDV,INHERENT_RISK_SCORE,year,BASE_AUM,TUOI,INCOME,CBAL,CBALORG,CBAL_AVG,CBAL_MAX,CBAL_MIN,AFLIMT_MAX,AFLIMT_MIN,AFLIMT_AVG,COLLATERAL_VALUE,LTV,N_AVG_DEPOSIT_12M,N_AVG_DD_12M,N_AVG_CD_12M,FLAG_SALARY_ACC,DURATION_MAX,REMAINING_DURATION_MAX,TIME_TO_OP_MAX,RATE_AVG,MAX_DPD_12M,MAX_DPD_12M_OBS,MAX_NHOMNOCIC,AVG_OD_DPD_12M,SUM_ALL_OD_12M,XULYNO,N_AVG_OVERDUE_CBAL_12M,BAD,BAD_NEXT_12M,N_AVG_DEPOSIT_3M,N_AVG_DEPOSIT_6M,N_AVG_DEPOSIT_9M,FLAG_DEPOSIT,REAL_GDP,REAL_GDP_GROWTH_12M,CPI,CPI_GROWTH_12M,UR,UR_GROWTH_12M,IIP,IIP_GROWTH_12M,CBAL_SHORTTERM_LOAN,CBAL_MIDTERM_LOAN,CBAL_LONGTERM_LOAN,HAS_SHORTTERM_LOAN,HAS_MIDTERM_LOAN,HAS_LONGTERM_LOAN,MAX_LTV_MO,MIN_LTV_MO,AVG_LTV_MO,CBAL_TO_INC_12MON,CBAL_TO_INC_9MON,CBAL_TO_INC_6MON,CBAL_TO_INC_3MON,INTEREST_12M,INTEREST,PRINPICAL_PYMT_FRQ_ID_MAX,N_PAYMENT_GOC_LAI,PURCOD_MAX,PURCOD_MIN,CBALORG_MAX,CBALORG_MIN,CBALORG_AVG,ROW_MISSING_PCT,FLAG_HIGH_MISSING
count,2.400000e+06,2.400000e+06,2.400000e+06,2.400000e+06,2.400000e+06,2.400000e+06,2.400000e+06,2.400000e+06,2.400000e+06,2.400000e+06,2.400000e+06,2.400000e+06,2.400000e+06,2.400000e+06,2.400000e+06,2.400000e+06,2.400000e+06,2.400000e+06,2.400000e+06,2.400000e+06,2.400000e+06,2.400000e+06,2.400000e+06,2.400000e+06,2.400000e+06,2.400000e+06,2.400000e+06,2.400000e+06,2.400000e+06,2.400000e+06,2.400000e+06,2.400000e+06,2.400000e+06,2.400000e+06,2.400000e+06,2.400000e+06,2.400000e+06,2.400000e+06,2.400000e+06,2.400000e+06,2.400000e+06,2.400000e+06,2.400000e+06,2.400000e+06,2.400000e+06,2.400000e+06,2.400000e+06,2.400000e+06,2.400000e+06,2.400000e+06,2.400000e+06,2.400000e+06,2.400000e+06,2.400000e+06,2.400000e+06,2.400000e+06,2.400000e+06,2.400000e+06,2.400000e+06,2.400000e+06,2.400000e+06,2.400000e+06,2.400000e+06,2.400000e+06,2.400000e+06,2.400000e+06,2.400000e+06,2.400000e+06,2.400000e+06,2.400000e+06,2.400000e+06,2400000.0,2400000.0
mean,1.200000e+06,3.449828e+01,1.855425e+00,3.375675e-01,1.020000e-02,-4.041076e-04,2.020500e+03,1.246821e+07,3.699782e+01,2.259495e+07,5.547822e+07,6.109716e+07,5.546875e+07,7.222598e+07,3.878468e+07,1.838248e+08,1.654424e+08,1.746336e+08,6.875032e+08,8.297120e+01,5.774457e+06,1.732337e+06,4.042120e+06,5.001950e-01,3.298405e+01,1.599313e+01,1.699127e+01,9.507676e+00,3.485155e+01,3.091566e+01,1.557207e+00,1.502981e+01,5.590107e+01,9.070000e-03,2.594170e+07,4.566167e-02,4.561250e-02,5.775815e+06,5.774690e+06,5.774537e+06,4.928550e-01,5.443333e+00,5.443691e+00,2.966667e+00,2.966645e+00,2.443333e+00,2.443315e+00,6.500000e+00,6.499777e+00,1.849903e+07,1.849663e+07,1.848257e+07,8.745337e-01,8.745337e-01,8.745337e-01,8.711974e+01,7.466695e+01,8.089334e+01,2.045332e+00,2.045415e+00,2.045427e+00,2.045330e+00,5.272267e+06,4.393552e+05,1.349517e+00,9.707726e+00,5.198849e+00,3.230792e+00,6.109716e+07,5.498744e+07,5.804230e+07,0.0,0.0
std,1.154701e+05,1.001233e+01,7.952914e-01,4.728803e-01,1.004787e-01,9.986586e-01,1.707825e+00,2.796085e+07,1.015738e+01,9.567950e+06,6.965422e+07,7.672316e+07,7.032108e+07,9.207899e+07,5.015759e+07,1.118754e+08,1.006878e+08,1.062816e+08,1.005835e+09,7.173200e+01,1.479381e+07,4.438143e+06,1.035567e+07,5.000001e-01,1.774346e+01,1.239425e+01,1.239581e+01,1.978125e+00,3.062356e+01,2.763187e+01,6.194110e-01,1.454765e+01,3.446064e+01,9.480369e-02,4.263153e+07,2.087503e-01,2.086433e-01,1.487844e+07,1.481832e+07,1.480114e+07,4.999491e-01,2.104556e+00,2.131550e+00,5.496263e-01,5.766827e-01,3.617858e-01,3.889085e-01,2.629956e+00,2.660330e+00,2.822275e+07,2.815987e+07,2.821687e+07,3.312469e-01,3.312469e-01,3.312469e-01,7.537465e+01,6.485327e+01,7.001533e+01,1.575752e+00,1.577640e+00,1.582839e+00,1.591470e+00,6.916356e+06,5.763629e+05,1.150762e+00,4.469618e+00,1.964701e+00,1.799335e+00,7.672316e+07,6.905084e+07,7.288700e+07,0.0,0.0
min,1.000000e+06,-1.300000e+01,1.000000e+00,0.000000e+00,0.000000e+00,-4.648333e+00,2.018000e+03,4.830000e+03,-1.300000e+01,7.542231e+06,0.000000e+00,3.062600e+04,0.000000e+00,0.000000e+00,0.000000e+00,4.026629e+07,3.623966e+07,3.82529

In [20]:
pd.set_option('display.max_columns', 20)

In [21]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2400000 entries, 0 to 2399999
Data columns (total 78 columns):
 #   Column                     Dtype  
---  ------                     -----  
 0   SOCIF                      int64  
 1   C_GIOITINH                 object 
 2   BASE_AGE                   int64  
 3   TRINHDO                    int64  
 4   TTHONNHAN                  object 
 5   SOHUUNHA                   int64  
 6   NHANVIENBIDV               int64  
 7   INHERENT_RISK_SCORE        float64
 8   year                       int64  
 9   BASE_AUM                   int64  
 10  final_CST_MKT_SEG          object 
 11  TUOI                       int64  
 12  INCOME                     int64  
 13  CBAL                       int64  
 14  CBALORG                    int64  
 15  CBAL_AVG                   int64  
 16  CBAL_MAX                   int64  
 17  CBAL_MIN                   int64  
 18  AFLIMT_MAX                 int64  
 19  AFLIMT_MIN                 int64  
 20  AF

In [22]:
cat_cols = df.select_dtypes(include=['object', 'category']).columns

for col in cat_cols:
    print(f"\n{'='*20} {col} {'='*20}")
    vc = df[col].value_counts(dropna=False)
    pct = df[col].value_counts(normalize=True, dropna=False) * 100
    print(pd.concat([vc, pct.rename('pct_%')], axis=1))


==================== C_GIOITINH ====================
              count     pct_%
C_GIOITINH                   
F           1176654  49.02725
M           1175634  48.98475
O             47712   1.98800

==================== TTHONNHAN ====================
             count     pct_%
TTHONNHAN                   
Single     1307310  54.47125
Married    1092690  45.52875

==================== final_CST_MKT_SEG ====================
                     count    pct_%
final_CST_MKT_SEG                  
Mass               2368812  98.7005
Upper                31092   1.2955
Private                 96   0.0040

==================== SAMPLE_TYPE ====================
               count      pct_%
SAMPLE_TYPE                    
TRAIN        1600000  66.666667
OOS           400000  16.666667
OOT           400000  16.666667


### Tính chính xác

Tuổi khách hàng

In [23]:
rule_acc_age = df['TUOI'] >= 15

LTV thực tế

In [24]:
rule_acc_ltv = df['LTV'] < 400

Hạn mức – dư nợ

In [25]:
rule_acc_limit = df['AFLIMT_MAX'] >= df['CBAL']

Thu nhập – dư nợ

In [26]:
rule_acc_dti = df['CBAL'] <= df['INCOME'] * 20

CIC – DPD

In [27]:
rule_acc_cic = ~(
    (df['MAX_DPD_12M_OBS'] < 30) & (df['MAX_NHOMNOCIC'] > 1)
) & ~(
    (df['MAX_DPD_12M_OBS'] >= 90) & (df['MAX_NHOMNOCIC'] < 3)
)

Flag & tỷ lệ chính xác

In [28]:
df['FLAG_INACCURATE'] = ~(
    rule_acc_age &
    rule_acc_ltv &
    rule_acc_limit &
    rule_acc_dti &
    rule_acc_cic
)

accuracy_rate = 1 - df['FLAG_INACCURATE'].mean()
accuracy_rate

np.float64(0.92909625)

In [29]:
rule_acc_age.sum()

np.int64(2367780)

In [30]:
rule_acc_ltv.sum()

np.int64(2399970)

In [31]:
rule_acc_dti.sum()

np.int64(2395873)

In [32]:
rule_acc_cic.sum()

np.int64(2264087)

### Tính đồng nhất

CBAL vs cấu trúc kỳ hạn

In [33]:
rule_cons_balance = (
    df['CBAL'] ==
    df['CBAL_SHORTTERM_LOAN']
    + df['CBAL_MIDTERM_LOAN']
    + df['CBAL_LONGTERM_LOAN']
)

df.loc[~rule_cons_balance].shape

(0, 79)

Deposit breakdown

In [34]:
rule_cons_deposit = (
    abs(
        df['N_AVG_DEPOSIT_12M'] -
        (df['N_AVG_DD_12M'] + df['N_AVG_CD_12M'])
    ) <= 1
)
df.loc[~rule_cons_deposit].shape

(0, 79)

DPD logic

In [35]:
rule_cons_dpd = df['MAX_DPD_12M_OBS'] <= df['MAX_DPD_12M']

df.loc[~rule_cons_dpd].shape


(0, 79)

Flag tổng hợp consistency

In [36]:
df['FLAG_INCONSISTENT'] = ~(
    rule_cons_balance &
    rule_cons_deposit &
    rule_cons_dpd
)

BẢNG TỔNG HỢP (RẤT NÊN CÓ)

In [37]:
quality_summary = pd.DataFrame({
    'Metric': [
        'Completeness',
        'Uniqueness',
        'Timeliness',
        'Validity',
        'Accuracy',
        'Consistency'
    ],
    'Issue_Rate_%': [
        df['FLAG_HIGH_MISSING'].mean() * 100,
        dup_full.mean() * 100,
        leak_check.shape[0] / len(df) * 100,
        df['FLAG_INVALID'].mean() * 100,
        df['FLAG_INACCURATE'].mean() * 100,
        df['FLAG_INCONSISTENT'].mean() * 100
    ]
})

quality_summary

,Metric,Issue_Rate_%
0,Completeness,0.000000
1,Uniqueness,0.000000
2,Timeliness,0.000000
3,Validity,0.008750
4,Accuracy,7.090375
5,Consistency,0.000000


In [38]:
df['FLAG_AGE_INVALID'] = (~rule_acc_age).astype(int)
df['FLAG_LTV_HIGH']    = (~rule_acc_ltv).astype(int)
df['FLAG_DTI_HIGH']    = (~rule_acc_dti).astype(int)
df['FLAG_CIC_LOW']     = (~rule_acc_cic).astype(int)
# df['FLAG_INCONSISTENT_DEPOSIT'] = (~rule_cons_deposit).astype(int)

In [39]:
import numpy as np

In [40]:
flag_cols = [c for c in df.columns if c.startswith('FLAG_')]

dq_flag_impact = []

for c in flag_cols:
    tmp = df.groupby(c)['BAD_NEXT_12M'].mean()
    dq_flag_impact.append({
        'FLAG': c,
        'BAD_RATE_0': tmp.get(0, np.nan),
        'BAD_RATE_1': tmp.get(1, np.nan),
        'LIFT': tmp.get(1, np.nan) / tmp.get(0, np.nan)
    })

dq_flag_impact = pd.DataFrame(dq_flag_impact).sort_values('LIFT', ascending=False)
dq_flag_impact


C:\Users\PC\AppData\Local\Temp\ipykernel_29964\4236309678.py:9: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  'BAD_RATE_0': tmp.get(0, np.nan),
C:\Users\PC\AppData\Local\Temp\ipykernel_29964\4236309678.py:10: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  'BAD_RATE_1': tmp.get(1, np.nan),
C:\Users\PC\AppData\Local\Temp\ipykernel_29964\4236309678.py:11: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  'LIFT': tmp.get(1, np.nan) / tmp.get

,FLAG,BAD_RATE_0,BAD_RATE_1,LIFT
3,FLAG_INVALID,0.045610,0.071429,1.566064
7,FLAG_LTV_HIGH,0.045612,0.066667,1.461596
6,FLAG_AGE_INVALID,0.045514,0.052855,1.161301
0,FLAG_SALARY_ACC,0.045492,0.045733,1.005300
1,FLAG_DEPOSIT,0.055172,0.035776,0.648454
8,FLAG_DTI_HIGH,0.045652,0.022535,0.493613
4,FLAG_INACCURATE,0.047706,0.018176,0.380999
9,FLAG_CIC_LOW,0.047771,0.009653,0.202073
2,FLAG_HIGH_MISSING,0.045613,NaN,NaN
5,FLAG_INCONSISTENT,0.045613,NaN,NaN


In [41]:
for f in ['FLAG_HIGH_MISSING', 'FLAG_INCONSISTENT']:
    print(f, df[f].value_counts(dropna=False))


FLAG_HIGH_MISSING FLAG_HIGH_MISSING
0    2400000
Name: count, dtype: int64
FLAG_INCONSISTENT FLAG_INCONSISTENT
False    2400000
Name: count, dtype: int64


In [42]:
df.shape

(2400000, 84)

In [43]:
df.drop(['FLAG_SALARY_ACC', 'FLAG_DTI_HIGH', 'FLAG_INACCURATE', 'FLAG_CIC_LOW', 'FLAG_HIGH_MISSING', 'FLAG_INCONSISTENT', 'ROW_MISSING_PCT'], axis=1, inplace=True)

In [44]:
df.columns

Index(['SOCIF', 'C_GIOITINH', 'BASE_AGE', 'TRINHDO', 'TTHONNHAN', 'SOHUUNHA',
       'NHANVIENBIDV', 'INHERENT_RISK_SCORE', 'year', 'BASE_AUM',
       'final_CST_MKT_SEG', 'TUOI', 'INCOME', 'CBAL', 'CBALORG', 'CBAL_AVG',
       'CBAL_MAX', 'CBAL_MIN', 'AFLIMT_MAX', 'AFLIMT_MIN', 'AFLIMT_AVG',
       'COLLATERAL_VALUE', 'LTV', 'N_AVG_DEPOSIT_12M', 'N_AVG_DD_12M',
       'N_AVG_CD_12M', 'DURATION_MAX', 'REMAINING_DURATION_MAX',
       'TIME_TO_OP_MAX', 'RATE_AVG', 'MAX_DPD_12M', 'MAX_DPD_12M_OBS',
       'MAX_NHOMNOCIC', 'AVG_OD_DPD_12M', 'SUM_ALL_OD_12M', 'XULYNO',
       'N_AVG_OVERDUE_CBAL_12M', 'BAD', 'BAD_NEXT_12M', 'N_AVG_DEPOSIT_3M',
       'N_AVG_DEPOSIT_6M', 'N_AVG_DEPOSIT_9M', 'FLAG_DEPOSIT', 'REAL_GDP',
       'REAL_GDP_GROWTH_12M', 'CPI', 'CPI_GROWTH_12M', 'UR', 'UR_GROWTH_12M',
       'IIP', 'IIP_GROWTH_12M', 'CBAL_SHORTTERM_LOAN', 'CBAL_MIDTERM_LOAN',
       'CBAL_LONGTERM_LOAN', 'HAS_SHORTTERM_LOAN', 'HAS_MIDTERM_LOAN',
       'HAS_LONGTERM_LOAN', 'MAX_LTV_MO', 'MIN_LTV_MO

In [45]:
df.drop('BAD', axis=1, inplace=True)

In [46]:
df_train = df[df['SAMPLE_TYPE'] == 'TRAIN']
df_train_y = df_train['BAD_NEXT_12M']
df_OOS = df[df['SAMPLE_TYPE'] == 'OOS']
df_OOS_y = df_OOS['BAD_NEXT_12M']
df_OOT = df[df['SAMPLE_TYPE'] == 'OOT']
df_OOT_y = df_OOT['BAD_NEXT_12M']

In [47]:
df_train.drop(['SAMPLE_TYPE','BAD_NEXT_12M'], axis=1, inplace=True)
df_OOS.drop(['SAMPLE_TYPE', 'BAD_NEXT_12M'], axis=1, inplace=True)
df_OOT.drop(['SAMPLE_TYPE', 'BAD_NEXT_12M'], axis=1, inplace=True)

C:\Users\PC\AppData\Local\Temp\ipykernel_29964\2937600189.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_train.drop(['SAMPLE_TYPE','BAD_NEXT_12M'], axis=1, inplace=True)
C:\Users\PC\AppData\Local\Temp\ipykernel_29964\2937600189.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_OOS.drop(['SAMPLE_TYPE', 'BAD_NEXT_12M'], axis=1, inplace=True)
C:\Users\PC\AppData\Local\Temp\ipykernel_29964\2937600189.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returni

In [48]:
df_train.columns

Index(['SOCIF', 'C_GIOITINH', 'BASE_AGE', 'TRINHDO', 'TTHONNHAN', 'SOHUUNHA',
       'NHANVIENBIDV', 'INHERENT_RISK_SCORE', 'year', 'BASE_AUM',
       'final_CST_MKT_SEG', 'TUOI', 'INCOME', 'CBAL', 'CBALORG', 'CBAL_AVG',
       'CBAL_MAX', 'CBAL_MIN', 'AFLIMT_MAX', 'AFLIMT_MIN', 'AFLIMT_AVG',
       'COLLATERAL_VALUE', 'LTV', 'N_AVG_DEPOSIT_12M', 'N_AVG_DD_12M',
       'N_AVG_CD_12M', 'DURATION_MAX', 'REMAINING_DURATION_MAX',
       'TIME_TO_OP_MAX', 'RATE_AVG', 'MAX_DPD_12M', 'MAX_DPD_12M_OBS',
       'MAX_NHOMNOCIC', 'AVG_OD_DPD_12M', 'SUM_ALL_OD_12M', 'XULYNO',
       'N_AVG_OVERDUE_CBAL_12M', 'N_AVG_DEPOSIT_3M', 'N_AVG_DEPOSIT_6M',
       'N_AVG_DEPOSIT_9M', 'FLAG_DEPOSIT', 'REAL_GDP', 'REAL_GDP_GROWTH_12M',
       'CPI', 'CPI_GROWTH_12M', 'UR', 'UR_GROWTH_12M', 'IIP', 'IIP_GROWTH_12M',
       'CBAL_SHORTTERM_LOAN', 'CBAL_MIDTERM_LOAN', 'CBAL_LONGTERM_LOAN',
       'HAS_SHORTTERM_LOAN', 'HAS_MIDTERM_LOAN', 'HAS_LONGTERM_LOAN',
       'MAX_LTV_MO', 'MIN_LTV_MO', 'AVG_LTV_MO', 'CBAL_

In [49]:
# df_train.to_csv('train.csv', index=False)
# df_OOS.to_csv('oos.csv', index=False)
# df_OOT.to_csv('oot.csv', index=False)

kiểm tra có leakage ko, nếu dùng toàn bộ data để phân khúc kh

In [ ]:
vars_finance = [
    'N_AVG_DEPOSIT_12M', 'N_AVG_DEPOSIT_3M', 'N_AVG_DEPOSIT_6M', 
    'N_AVG_DEPOSIT_9M', 'FLAG_DEPOSIT'
]

# Nhóm 2: Thông tin dư nợ (Khoản vay)
vars_loan = [
    'CBAL', 'CBAL_AVG', 'CBAL_MAX', 'CBAL_MIN', 
    'CBAL_SHORTTERM_LOAN', 'CBAL_MIDTERM_LOAN', 'CBAL_LONGTERM_LOAN',
    'TOTAL_DEBT' # Nếu có biến tổng dư nợ
]

# (Lưu ý: Chỉ chọn các biến có trong file dữ liệu của bạn, nếu biến nào không có thì xóa khỏi list trên)
# Để demo mình sẽ chạy chung tất cả các biến bạn đã liệt kê ở câu trước mà có tính chất số học
all_features = [
    'N_AVG_DEPOSIT_12M', 'N_AVG_DEPOSIT_3M', 'N_AVG_DEPOSIT_6M', 'N_AVG_DEPOSIT_9M',
    'CBAL', 'CBAL_AVG', 'CBAL_MAX', 'CBAL_MIN', 
    'AFLIMT_MAX', 'AFLIMT_AVG', 'COLLATERAL_VALUE'
]

# Kiểm tra xem biến nào thực sự tồn tại trong dataframe
valid_features = [col for col in all_features if col in df.columns]
target = 'BAD_NEXT_12M'

# 3. Xử lý dữ liệu (Cây quyết định của Sklearn không chịu được Null/NaN)
X = df[valid_features].copy()
y = df[target]

# Điền giá trị 0 vào các ô trống (hoặc dùng trung bình tùy nghiệp vụ)
imputer = SimpleImputer(strategy='constant', fill_value=0)                       # ko có null

X_filled = imputer.fit_transform(X)

# 4. Cấu hình Cây quyết định (Theo đúng Báo cáo BIDV)
# criterion='gini', class_weight='balanced'
dt_model = DecisionTreeClassifier(
    criterion='gini',
    splitter='best',
    class_weight='balanced',
    random_state=42,
    max_depth=5  # Giới hạn độ sâu để tránh overfitting khi tính độ quan trọng
)

# Huấn luyện mô hình
dt_model.fit(X_filled, y)

# 5. Trích xuất Feature Importance
importance_df = pd.DataFrame({
    'Tên Biến': valid_features,
    'Độ Quan Trọng (Feature Importance)': dt_model.feature_importances_
})

# Sắp xếp từ cao xuống thấp
importance_df = importance_df.sort_values(by='Độ Quan Trọng (Feature Importance)', ascending=False)

print("--- KẾT QUẢ XẾP HẠNG ĐỘ QUAN TRỌNG CỦA BIẾN ---")
print(importance_df)

In [ ]:
import pandas as pd
from optbinning import OptimalBinning

# --- BƯỚC 1: CHUẨN BỊ DỮ LIỆU ---
# (Đảm bảo biến 'df' của bạn đã có sẵn dữ liệu và cột 'BAD_NEXT_12M')
# Lưu ý: Optbinning cần xử lý giá trị Null, ta tạm thời điền 0 nếu có Null
df_clean = df.copy()
df_clean['N_AVG_DEPOSIT_12M'] = df_clean['N_AVG_DEPOSIT_12M'].fillna(0)
df_clean['CBAL'] = df_clean['CBAL'].fillna(0)

# --- BƯỚC 2: TÌM ĐIỂM CẮT CHO TIỀN GỬI (N_AVG_DEPOSIT_12M) ---
print(">>> ĐANG TÍNH TOÁN NGƯỠNG CẮT CHO: TIỀN GỬI 12 THÁNG...")

# Cấu hình: max_n_bins=2 (Chia làm 2 nhóm: Thấp/Cao)
opt_deposit = OptimalBinning(name="N_AVG_DEPOSIT_12M", dtype="numerical", solver="cp",
                             max_n_bins=2, min_bin_size=0.1)

opt_deposit.fit(df_clean['N_AVG_DEPOSIT_12M'], df_clean['BAD_NEXT_12M'])

print(f"Trạng thái: {opt_deposit.status}")
print(f"Điểm cắt gợi ý (VNĐ): {opt_deposit.splits}")

# Xem bảng chi tiết (IV, WoE)
print("\n--- BẢNG THỐNG KÊ PHÂN KHÚC TIỀN GỬI ---")
display(opt_deposit.binning_table.build()[['Bin', 'Count', 'Event', 'WoE', 'IV']])


# --- BƯỚC 3: TÌM ĐIỂM CẮT CHO DƯ NỢ (CBAL) ---
print("\n\n>>> ĐANG TÍNH TOÁN NGƯỠNG CẮT CHO: DƯ NỢ HIỆN TẠI (CBAL)...")

opt_loan = OptimalBinning(name="CBAL", dtype="numerical", solver="cp",
                          max_n_bins=2, min_bin_size=0.1)

opt_loan.fit(df_clean['CBAL'], df_clean['BAD_NEXT_12M'])

print(f"Trạng thái: {opt_loan.status}")
print(f"Điểm cắt gợi ý (VNĐ): {opt_loan.splits}")

# Xem bảng chi tiết
print("\n--- BẢNG THỐNG KÊ PHÂN KHÚC DƯ NỢ ---")
display(opt_loan.binning_table.build()[['Bin', 'Count', 'Event', 'WoE', 'IV']])

In [ ]:
import numpy as np

# 1. Đặt ngưỡng làm tròn (Business Rule)
CUTOFF_DEPOSIT = 1500000  # 1.5 Triệu
CUTOFF_LOAN = 40000000    # 40 Triệu

# 2. Hàm định nghĩa phân khúc
def define_segment(row):
    # Logic: Nếu Tiền gửi thấp
    if row['N_AVG_DEPOSIT_12M'] < CUTOFF_DEPOSIT:
        if row['CBAL'] < CUTOFF_LOAN:
            return 'SEG1_LowDep_LowLoan'
        else:
            return 'SEG2_LowDep_HighLoan'
    # Logic: Nếu Tiền gửi cao
    else:
        if row['CBAL'] < CUTOFF_LOAN:
            return 'SEG3_HighDep_LowLoan'
        else:
            return 'SEG4_HighDep_HighLoan'

# 3. Áp dụng vào Dataframe
# Giả sử df là dataframe của bạn
df['FINAL_SEGMENT'] = df.apply(define_segment, axis=1)

# 4. Kiểm tra số lượng và tỷ lệ nợ xấu từng phân khúc
summary = df.groupby('FINAL_SEGMENT').agg(
    So_Luong=('BAD_NEXT_12M', 'count'),
    So_Bad=('BAD_NEXT_12M', 'sum')
).reset_index()

summary['Bad_Rate'] = (summary['So_Bad'] / summary['So_Luong']) * 100

print("--- KẾT QUẢ PHÂN KHÚC CUỐI CÙNG ---")
print(summary)

In [ ]:
# 1. Lưu kết quả phân khúc vào file để dùng lâu dài
# df.to_csv('Dulieu_DaPhanKhuc.csv', index=False)

# 2. Tách dữ liệu để xây dựng mô hình cho SEGMENT 1 (Rủi ro cao nhất)
print("--- CHUẨN BỊ DỮ LIỆU CHO SEGMENT 1 ---")

# Lọc lấy riêng SEG1
df_seg1 = df[df['FINAL_SEGMENT'] == 'SEG1_LowDep_LowLoan'].copy()

print(f"Dữ liệu Segment 1: {df_seg1.shape}")
print(f"Tỷ lệ Bad: {(df_seg1['BAD_NEXT_12M'].sum() / len(df_seg1))*100:.2f}%")

# 3. Chia tập Train/Test (OOS) cho riêng Segment 1
from sklearn.model_selection import train_test_split

X_seg1 = df_seg1.drop(columns=['BAD_NEXT_12M', 'FINAL_SEGMENT']) # Bỏ cột target và nhãn phân khúc
y_seg1 = df_seg1['BAD_NEXT_12M']

# Chia 70% Train - 30% Test
X_train, X_test, y_train, y_test = train_test_split(X_seg1, y_seg1, test_size=0.3, random_state=42, stratify=y_seg1)

print(f"Số lượng tập Train: {len(X_train)}")
print(f"Số lượng tập Test: {len(X_test)}")

# --- TỪ ĐÂY BẮT ĐẦU BƯỚC BINNING (WOE/IV) CHO SEGMENT 1 ---
# Bạn sẽ lặp lại quy trình OptBinning cho các biến trong X_train để tìm các biến tốt nhất cho nhóm này.

In [ ]:
import pandas as pd
import numpy as np
from optbinning import OptimalBinning
import warnings

# Tắt các cảnh báo "FutureWarning" cho đỡ rối mắt
warnings.filterwarnings("ignore")

# 1. Danh sách loại bỏ (giữ nguyên logic của bạn)
drop_list = [
    'BAD_NEXT_12M', 'BAD', 'XULYNO', 'MAX_NHOMNOCIC', 'INHERENT_RISK_SCORE', # Target & Leakage
    'SOCIF', 'year', 'SAMPLE_TYPE', 'final_CST_MKT_SEG', 'SEGMENT_DEPOSIT', 'SEGMENT_LOAN', # ID & Admin
    'REAL_GDP', 'REAL_GDP_GROWTH_12M', 'CPI', 'CPI_GROWTH_12M', 
    'UR', 'UR_GROWTH_12M', 'IIP', 'IIP_GROWTH_12M', # Macro
    'TUOI', 'CIF_ID', 'DATE', 'REPORT_DATE' # Trùng lặp & ID khác
]

# Lọc danh sách biến thực tế có trong X_train
final_features = [col for col in X_train.columns if col not in drop_list]

print(f"Tổng số biến sẽ chạy IV: {len(final_features)}")

# 2. Chuẩn bị DataFrame lưu kết quả
iv_summary = []

print("--- ĐANG TÍNH IV (Sử dụng Solver='ls' để ổn định hơn) ---")

for col in final_features:
    try:
        # --- BƯỚC XỬ LÝ QUAN TRỌNG ---
        # 1. Kiểm tra biến hằng số (chỉ có 1 giá trị) -> Bỏ qua vì không tính được IV
        if X_train[col].nunique() <= 1:
            print(f"  -> Bỏ qua {col}: Chỉ có 1 giá trị duy nhất.")
            continue

        # 2. Xác định kiểu dữ liệu chuẩn xác
        if pd.api.types.is_numeric_dtype(X_train[col]):
            col_type = "numerical"
            # Chuyển về dạng số thực, thay thế vô cùng (inf) bằng NaN rồi điền 0 hoặc trung vị
            x_data = X_train[col].replace([np.inf, -np.inf], np.nan).fillna(0)
        else:
            col_type = "categorical"
            # Chuyển hết về dạng chuỗi để tránh lỗi object lẫn lộn
            x_data = X_train[col].astype(str).fillna("MISSING")

        # 3. Chạy OptBinning với solver="ls"
        optb = OptimalBinning(
            name=col, 
            dtype=col_type, 
            solver="ls",  # <--- THAY ĐỔI QUAN TRỌNG: Dùng Local Search thay vì CP
            max_n_bins=5, 
            min_bin_size=0.05
        )
        
        optb.fit(x_data, y_train)
        
        # 4. Lưu kết quả nếu chạy thành công
        if optb.status in ["OPTIMAL", "FEASIBLE"]:
            iv_value = optb.binning_table.iv
            iv_summary.append({
                'Variable': col,
                'IV': iv_value,
                'Type': col_type,
                'Status': optb.status
            })
        else:
            print(f"  -> {col}: Không tìm được điểm cắt tối ưu (Status: {optb.status})")

    except Exception as e:
        # In lỗi gọn gàng hơn
        print(f"  [!] Lỗi biến {col}: {str(e)}")

# 3. Tổng hợp và Hiển thị kết quả
if len(iv_summary) > 0:
    df_iv = pd.DataFrame(iv_summary)
    df_iv = df_iv.sort_values(by='IV', ascending=False).reset_index(drop=True)

    # Phân loại sức mạnh
    def classify_iv(iv):
        if iv < 0.02: return 'Yếu (Loại)'
        elif iv < 0.1: return 'Trung bình'
        elif iv < 0.3: return 'Khá'
        elif iv < 0.5: return 'Mạnh'
        else: return 'Rất mạnh (Check Leakage)'

    df_iv['Strength'] = df_iv['IV'].apply(classify_iv)

    print("\n" + "="*50)
    print("TOP 20 BIẾN CÓ IV CAO NHẤT")
    print("="*50)
    display(df_iv.head(20))
    
    # Xuất ra list biến đạt chuẩn để dùng tiếp
    selected_vars = df_iv[df_iv['IV'] >= 0.02]['Variable'].tolist()
    print(f"\n>> Số lượng biến đạt chuẩn (IV >= 0.02): {len(selected_vars)} biến.")
else:
    print("\nKhông tính được IV cho biến nào cả. Hãy kiểm tra lại dữ liệu đầu vào!")

In [ ]:
import pandas as pd
import numpy as np
import warnings

warnings.filterwarnings("ignore")

# --- HÀM TÍNH IV THỦ CÔNG ---
def calculate_iv_manual(df, feature, target):
    lst = []
    df[feature] = df[feature].fillna("MISSING")
    
    # 1. Xử lý chia nhóm (Binning)
    try:
        if pd.api.types.is_numeric_dtype(df[feature]):
            # Nếu là số: Chia thành 10 phần bằng nhau (Deciles) hoặc ít hơn nếu dữ liệu ít
            # duplicates='drop' để gộp các nhóm trùng nhau (ví dụ quá nhiều số 0)
            df['bin'] = pd.qcut(df[feature], q=10, duplicates='drop').astype(str)
        else:
            # Nếu là chữ: Giữ nguyên nhóm, nếu quá nhiều nhóm nhỏ thì gộp vào 'OTHER'
            # Ở đây ta làm đơn giản là giữ nguyên
            df['bin'] = df[feature].astype(str)
    except Exception as e:
        # Nếu lỗi qcut (ví dụ biến chỉ có 1 giá trị), coi như 1 bin
        df['bin'] = "Single_Bin"

    # 2. Tính toán Good/Bad
    # Tổng số Bad và Good toàn tập
    total_bad = df[target].sum()
    total_good = df[target].count() - total_bad
    
    # Group by theo bin
    grouped = df.groupby('bin', as_index=False).agg({target: ['count', 'sum']})
    grouped.columns = ['Bin', 'Total', 'Bad']
    grouped['Good'] = grouped['Total'] - grouped['Bad']
    
    # 3. Tính WOE và IV
    # Thêm 0.5 vào để tránh lỗi chia cho 0 (Laplace smoothing)
    grouped['Dist_Bad'] = (grouped['Bad'] + 0.5) / (total_bad + 0.5)
    grouped['Dist_Good'] = (grouped['Good'] + 0.5) / (total_good + 0.5)
    
    grouped['WoE'] = np.log(grouped['Dist_Good'] / grouped['Dist_Bad'])
    grouped['IV_Component'] = (grouped['Dist_Good'] - grouped['Dist_Bad']) * grouped['WoE']
    
    # Tổng IV của biến
    iv_sum = grouped['IV_Component'].sum()
    
    return iv_sum

# --- CHẠY VÒNG LẶP CHO DANH SÁCH BIẾN CỦA BẠN ---

# 1. Chuẩn bị danh sách biến (Lấy từ code trước của bạn)
drop_list = [
    'BAD_NEXT_12M', 'BAD', 'XULYNO', 'MAX_NHOMNOCIC', 'INHERENT_RISK_SCORE',
    'SOCIF', 'year', 'SAMPLE_TYPE', 'final_CST_MKT_SEG', 'SEGMENT_DEPOSIT', 'SEGMENT_LOAN',
    'REAL_GDP', 'REAL_GDP_GROWTH_12M', 'CPI', 'CPI_GROWTH_12M', 
    'UR', 'UR_GROWTH_12M', 'IIP', 'IIP_GROWTH_12M',
    'TUOI', 'CIF_ID', 'DATE', 'REPORT_DATE'
]

# Lọc biến có trong X_train
final_features = [col for col in X_train.columns if col not in drop_list]

print(f"--- BẮT ĐẦU TÍNH IV CHO {len(final_features)} BIẾN (PHƯƠNG PHÁP PANDAS) ---")

iv_results = []

# Sử dụng copy để không ảnh hưởng dữ liệu gốc
df_calc = X_train[final_features].copy()
df_calc['target'] = y_train.values # Gắn target vào để tính

for col in final_features:
    try:
        # Bỏ qua biến chỉ có 1 giá trị
        if df_calc[col].nunique() <= 1:
            continue
            
        iv = calculate_iv_manual(df_calc, col, 'target')
        
        iv_results.append({
            'Variable': col,
            'IV': iv
        })
    except Exception as e:
        print(f"Lỗi biến {col}: {e}")

# --- HIỂN THỊ KẾT QUẢ ---
if len(iv_results) > 0:
    df_iv = pd.DataFrame(iv_results)
    df_iv = df_iv.sort_values(by='IV', ascending=False).reset_index(drop=True)

    def classify_iv(iv):
        if iv < 0.02: return 'Yếu (Loại)'
        elif iv < 0.1: return 'Trung bình'
        elif iv < 0.3: return 'Khá'
        elif iv < 0.5: return 'Mạnh'
        else: return 'Rất mạnh (Check Leakage)'

    df_iv['Strength'] = df_iv['IV'].apply(classify_iv)

    print("\n" + "="*50)
    print("TOP 20 BIẾN CÓ IV CAO NHẤT (Ranking)")
    print("="*50)
    display(df_iv.head(20))
    
    # Lấy danh sách biến tốt
    selected_vars = df_iv[df_iv['IV'] >= 0.02]['Variable'].tolist()
    print(f"\n>> Số lượng biến đạt chuẩn (IV >= 0.02): {len(selected_vars)} biến.")
else:
    print("Không tính được IV nào.")

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# 1. Lọc lấy danh sách các biến có IV >= 0.02
high_iv_vars = df_iv[df_iv['IV'] >= 0.02]['Variable'].tolist()

print(f"Số biến đạt chuẩn IV (>= 0.02): {len(high_iv_vars)}")
print(f"Danh sách: {high_iv_vars}")

# 2. Tính ma trận tương quan trên tập Train
# Chỉ lấy các biến số (Numerical) để tính correlation
numeric_cols = X_train[high_iv_vars].select_dtypes(include=[np.number]).columns.tolist()
corr_matrix = X_train[numeric_cols].corr().abs()

# 3. Vẽ Heatmap để bạn dễ nhìn (Optional)
plt.figure(figsize=(12, 10))
sns.heatmap(corr_matrix, annot=False, cmap='coolwarm')
plt.title("Ma trận tương quan giữa các biến đạt chuẩn")
plt.show()

# 4. Thuật toán tự động loại bỏ biến tương quan cao (> 0.7)
# Giữ lại biến có IV cao hơn
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))
to_drop = []

print("\n--- PHÂN TÍCH LOẠI BỎ BIẾN TƯƠNG QUAN CAO (> 0.7) ---")

for column in upper.columns:
    if any(upper[column] > 0.7): # Ngưỡng cắt correlation là 0.7
        # Tìm các biến có tương quan với cột này
        correlated_vars = upper.index[upper[column] > 0.7].tolist()
        
        for var in correlated_vars:
            # So sánh IV để quyết định giữ biến nào
            iv_col = df_iv.loc[df_iv['Variable'] == column, 'IV'].values[0]
            iv_var = df_iv.loc[df_iv['Variable'] == var, 'IV'].values[0]
            
            if iv_col < iv_var:
                mark_drop = column
                keep = var
            else:
                mark_drop = var
                keep = column
            
            if mark_drop not in to_drop:
                to_drop.append(mark_drop)
                print(f"  > Phát hiện cặp ({column}, {var}) tương quan cao.")
                print(f"    -> Giữ lại: {keep} (IV={max(iv_col, iv_var):.4f})")
                print(f"    -> Loại bỏ: {mark_drop} (IV={min(iv_col, iv_var):.4f})")

# 5. Chốt danh sách cuối cùng
final_vars_for_model = [x for x in high_iv_vars if x not in to_drop]

print("\n" + "="*50)
print(f"DANH SÁCH BIẾN CUỐI CÙNG ĐỂ CHẠY MÔ HÌNH ({len(final_vars_for_model)} biến)")
print("="*50)
print(final_vars_for_model)

In [ ]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
from sklearn.metrics import roc_auc_score
import matplotlib.pyplot as plt
import warnings

warnings.filterwarnings("ignore")

# 1. Danh sách 5 biến vàng đã chọn
final_vars = ['MAX_DPD_12M', 'BASE_AUM', 'N_AVG_OVERDUE_CBAL_12M', 'COLLATERAL_VALUE', 'N_AVG_DEPOSIT_6M']

# 2. Hàm tự tính WoE (ĐÃ SỬA LỖI)
def manual_woe_transform(feature_data, target_data):
    """
    feature_data: Series chứa dữ liệu biến cần biến đổi
    target_data: Series chứa dữ liệu target (0/1)
    """
    # Tạo dataframe tạm để xử lý
    temp_df = pd.DataFrame()
    temp_df['val'] = feature_data.values
    temp_df['target'] = target_data.values
    
    # Xử lý binning (Chia nhóm)
    try:
        if pd.api.types.is_numeric_dtype(feature_data):
            # Xử lý vô cùng và NaN
            col_clean = temp_df['val'].replace([np.inf, -np.inf], np.nan).fillna(0)
            
            # Nếu biến có quá ít giá trị unique (< 10), coi như là biến phân loại (categorical)
            if col_clean.nunique() < 10:
                 temp_df['bin'] = col_clean.astype(str)
            else:
                # Chia thành 5 nhóm (Quintiles)
                # Dùng duplicates='drop' để gộp các nhóm trùng nhau (ví dụ quá nhiều số 0)
                temp_df['bin'] = pd.qcut(col_clean, q=5, duplicates='drop').astype(str)
        else:
            temp_df['bin'] = temp_df['val'].astype(str).fillna("MISSING")
    except Exception as e:
        # Fallback nếu lỗi qcut: gom hết vào 1 nhóm (không khuyến khích nhưng để code chạy thông)
        temp_df['bin'] = "Single_Bin"

    # Tính toán WoE
    # Tổng Bad/Good toàn mẫu
    total_bad = temp_df['target'].sum()
    total_good = temp_df['target'].count() - total_bad
    
    # Group by bin
    grouped = temp_df.groupby('bin', as_index=False).agg({'target': ['count', 'sum']})
    grouped.columns = ['bin', 'total', 'bad']
    grouped['good'] = grouped['total'] - grouped['bad']
    
    # Tính tỷ lệ phân phối & WoE
    # Thêm 0.5 (Laplace Smoothing) để tránh lỗi chia cho 0
    grouped['dist_bad'] = (grouped['bad'] + 0.5) / (total_bad + 0.5)
    grouped['dist_good'] = (grouped['good'] + 0.5) / (total_good + 0.5)
    grouped['WoE'] = np.log(grouped['dist_good'] / grouped['dist_bad'])
    
    # Tạo từ điển mapping
    woe_map = dict(zip(grouped['bin'], grouped['WoE']))
    
    # Map giá trị WoE trả lại
    return temp_df['bin'].map(woe_map).values

# 3. Thực hiện biến đổi dữ liệu sang WoE
print("--- ĐANG BIẾN ĐỔI DỮ LIỆU SANG WOE ---")

X_train_woe = pd.DataFrame()

# Copy index từ y_train để đảm bảo khớp dòng
X_train_woe.index = y_train.index 

for col in final_vars:
    try:
        # Truyền trực tiếp Series X và Series y vào hàm
        X_train_woe[col] = manual_woe_transform(X_train[col], y_train)
        print(f"  > Đã biến đổi xong: {col}")
    except Exception as e:
        print(f"  [!] Lỗi biến {col}: {str(e)}")

# 4. Chạy Hồi quy Logistic (Scorecard Model)
print("\n--- KẾT QUẢ MÔ HÌNH LOGISTIC REGRESSION ---")

# Thêm cột hằng số (Intercept)
X_log = sm.add_constant(X_train_woe)
y_log = y_train.values

# Huấn luyện mô hình
try:
    model = sm.Logit(y_log, X_log).fit(disp=0)
    
    # Hiển thị bảng thống kê
    print(model.summary())
    
    # 5. Đánh giá Gini
    y_pred_prob = model.predict(X_log)
    auc = roc_auc_score(y_log, y_pred_prob)
    gini = 2 * auc - 1
    print(f"\n>> GINI TRÊN TẬP TRAIN: {gini:.2%} (Thang đo 0-100%)")
    
    # 6. Chuyển đổi sang điểm số (Score Scaling)
    pdo = 20
    base_score = 600
    base_odds = 50
    factor = pdo / np.log(2)
    offset = base_score - (factor * np.log(base_odds))
    
    df_results = pd.DataFrame()
    df_results['Log_Odds'] = model.predict(X_log, transform=False)
    
    # Công thức tính điểm: Score = Offset + Factor * (-Log_Odds)
    # Lưu ý: WoE càng cao -> Rủi ro càng thấp -> Log_Odds (p/1-p) càng thấp
    df_results['Score'] = offset - (factor * df_results['Log_Odds'])
    df_results['Score'] = df_results['Score'].clip(300, 850).astype(int)
    
    # Xem phân phối điểm
    plt.figure(figsize=(10, 6))
    plt.hist(df_results[y_log==0]['Score'], bins=30, alpha=0.5, label='Good', color='green')
    plt.hist(df_results[y_log==1]['Score'], bins=30, alpha=0.5, label='Bad', color='red')
    plt.title(f"Phân phối điểm tín dụng (Gini = {gini:.1%})")
    plt.xlabel("Điểm")
    plt.ylabel("Số lượng KH")
    plt.legend()
    plt.show()
    
except Exception as e:
    print(f"Lỗi khi chạy mô hình: {str(e)}")

In [ ]:
# 1. Cập nhật danh sách biến (Loại bỏ COLLATERAL_VALUE)
final_vars_clean = ['MAX_DPD_12M', 'BASE_AUM', 'N_AVG_OVERDUE_CBAL_12M', 'N_AVG_DEPOSIT_6M']

print(f"--- CHẠY MÔ HÌNH LẦN CUỐI VỚI {len(final_vars_clean)} BIẾN ---")

# 2. Chuẩn bị dữ liệu WoE cho các biến đã chọn
# (Lưu ý: Tận dụng lại hàm manual_woe_transform ở bước trước)
X_train_final = pd.DataFrame()
# Copy index để đảm bảo khớp
X_train_final.index = y_train.index 

for col in final_vars_clean:
    X_train_final[col] = manual_woe_transform(X_train[col], y_train)

# 3. Chạy Hồi quy Logistic
X_log_final = sm.add_constant(X_train_final)
y_log = y_train.values

model_final = sm.Logit(y_log, X_log_final).fit(disp=0)

# 4. Kiểm tra lại P-value và Gini
print(model_final.summary())

y_pred_final = model_final.predict(X_log_final)
auc_final = roc_auc_score(y_log, y_pred_final)
gini_final = 2 * auc_final - 1

print(f"\n>> GINI CUỐI CÙNG: {gini_final:.2%}")

# 5. Xuất Thẻ điểm (Scorecard Table)
# Đây là bảng quan trọng nhất để đưa vào hệ thống code
print("\n--- BẢNG THẺ ĐIỂM (SCORECARD) ---")

scorecard_df = pd.DataFrame()
pdo = 20
base_score = 600
base_odds = 50
factor = pdo / np.log(2)
offset = base_score - (factor * np.log(base_odds))

# Lấy hệ số (Beta) từ mô hình
intercept = model_final.params['const']
features_beta = model_final.params.drop('const')

# Tính điểm cơ bản (Base Score) phân bổ cho Intercept
# Công thức: (Intercept/n_features * Factor) + (Offset/n_features)
# Tuy nhiên để đơn giản, ta thường cộng hết vào 1 dòng Base Score hoặc chia đều.
# Cách hiển thị chuẩn:
print(f"Intercept (A0): {intercept:.4f}")
print(f"Factor: {factor:.4f}")
print(f"Offset: {offset:.4f}")

print("\nCông thức tính điểm từng biến:")
for col in final_vars_clean:
    beta = features_beta[col]
    print(f"  Biến {col}: Score = ({beta:.4f} * WoE) * {-factor:.4f}")

# 6. Biểu đồ phân phối điểm cuối cùng
df_final_score = pd.DataFrame()
df_final_score['Score'] = offset - factor * (intercept + np.dot(X_train_final.values, features_beta.values))
df_final_score['Score'] = df_final_score['Score'].clip(300, 850).astype(int)

plt.figure(figsize=(10, 6))
sns.kdeplot(df_final_score[y_log==0]['Score'], label='Good', shade=True, color='green')
sns.kdeplot(df_final_score[y_log==1]['Score'], label='Bad', shade=True, color='red')
plt.title(f'Phân phối điểm (Gini Final = {gini_final:.1%})')
plt.xlabel('Credit Score')
plt.legend()
plt.show()

In [ ]:
import pandas as pd
import numpy as np

# --- CẤU HÌNH THAM SỐ TỪ KẾT QUẢ MÔ HÌNH CỦA BẠN ---
final_vars_clean = ['MAX_DPD_12M', 'BASE_AUM', 'N_AVG_OVERDUE_CBAL_12M', 'N_AVG_DEPOSIT_6M']
coef_dict = {
    'MAX_DPD_12M': -0.9892,
    'BASE_AUM': -0.1850,
    'N_AVG_OVERDUE_CBAL_12M': -0.0633,
    'N_AVG_DEPOSIT_6M': -0.2119
}
intercept = -2.7135
factor = 28.8539
offset = 487.1229

# Tính điểm cơ sở (Base Score) từ hằng số
# Điểm này sẽ được cộng thẳng cho mọi khách hàng
intercept_points = offset - (factor * intercept)
print(f"--- ĐIỂM CƠ SỞ (BASE POINTS): {int(intercept_points)} ---")

# --- HÀM TẠO BẢNG ĐIỂM CHI TIẾT ---
scorecard_rows = []

print("\n--- ĐANG TẠO BẢNG TRA ĐIỂM... ---")

for feature in final_vars_clean:
    # Lấy lại dữ liệu gốc và target
    temp_df = pd.DataFrame()
    temp_df['val'] = X_train[feature].replace([np.inf, -np.inf], np.nan).fillna(0)
    temp_df['target'] = y_train.values
    
    # Binning lại (Logic giống hệt lúc train)
    if pd.api.types.is_numeric_dtype(temp_df['val']) and temp_df['val'].nunique() >= 10:
        temp_df['bin'] = pd.qcut(temp_df['val'], q=5, duplicates='drop').astype(str)
    else:
        temp_df['bin'] = temp_df['val'].astype(str).fillna("MISSING")
    
    # Tính WoE từng bin
    total_bad = temp_df['target'].sum()
    total_good = temp_df['target'].count() - total_bad
    
    grouped = temp_df.groupby('bin', as_index=False).agg({'target': ['count', 'sum']})
    grouped.columns = ['Bin_Range', 'Total', 'Bad']
    grouped['Good'] = grouped['Total'] - grouped['Bad']
    
    # Tính WoE
    grouped['Dist_Bad'] = (grouped['Bad'] + 0.5) / (total_bad + 0.5)
    grouped['Dist_Good'] = (grouped['Good'] + 0.5) / (total_good + 0.5)
    grouped['WoE'] = np.log(grouped['Dist_Good'] / grouped['Dist_Bad'])
    
    # TÍNH ĐIỂM (POINT) CHO TỪNG BIN
    # Point = WoE * Coef * (-Factor)
    beta = coef_dict[feature]
    grouped['Points'] = grouped['WoE'] * beta * (-factor)
    grouped['Points'] = grouped['Points'].round(0).astype(int) # Làm tròn điểm
    
    # Thêm thông tin vào list tổng
    for _, row in grouped.iterrows():
        scorecard_rows.append({
            'Variable': feature,
            'Bin_Range': row['Bin_Range'],
            'Count': row['Total'],
            'Bad_Rate': f"{(row['Bad']/row['Total'])*100:.2f}%",
            'WoE': round(row['WoE'], 4),
            'Coefficients': beta,
            'Score_Points': row['Points']
        })

# --- HIỂN THỊ VÀ XUẤT FILE ---
final_card = pd.DataFrame(scorecard_rows)

# Sắp xếp cho đẹp
final_card = final_card[['Variable', 'Bin_Range', 'Count', 'Bad_Rate', 'WoE', 'Coefficients', 'Score_Points']]

print("\n=== KẾT QUẢ THẺ ĐIỂM (MẪU) ===")
display(final_card)

# Lưu ra Excel để báo cáo sếp
# final_card.to_excel("Final_Scorecard_Model.xlsx", index=False)
# print("\nĐã xuất file Excel thành công!")

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# --- CẤU HÌNH LẠI THAM SỐ (Lấy từ kết quả mô hình đã chạy) ---
# Đảm bảo các biến model_final, X_log_final, y_train vẫn còn trong bộ nhớ
# Nếu mất, bạn cần chạy lại bước Hồi quy Logistic bên trên.

pdo = 20
base_score = 600
base_odds = 50
factor = pdo / np.log(2)
offset = base_score - (factor * np.log(base_odds))

print("--- ĐANG TÍNH TOÁN LẠI BẢNG ĐIỂM VÀ XẾP HẠNG ---")

# 1. Tạo lại DataFrame kết quả từ đầu (Fix lỗi thiếu cột)
df_results = pd.DataFrame()

# Lấy giá trị Log-odds dự báo từ mô hình
df_results['Log_Odds'] = model_final.predict(X_log_final, transform=False)

# Tính điểm Score
df_results['Score'] = offset - (factor * df_results['Log_Odds'])
df_results['Score'] = df_results['Score'].clip(300, 850).astype(int) # Làm tròn và chặn trần/sàn

# QUAN TRỌNG: Gán lại cột Target (Biến y_train) để fix lỗi KeyError
df_results['Target'] = y_train.values 

# 2. Phân hạng tín dụng (Rating)
# Dùng cut (chia đều điểm) thay vì qcut để tránh lỗi trùng điểm
# Chia dải điểm thành 10 hạng
df_results['Rating_Class'] = pd.cut(df_results['Score'], bins=10, labels=False)

# Đảo ngược nhãn: Hạng 9 (điểm cao nhất) -> AAA, Hạng 0 -> D
# Vì pd.cut đánh số 0 cho bin điểm thấp nhất -> Đúng logic 0=D, 9=AAA
rating_map = {
    0: 'D (Rủi ro cao nhất)', 
    1: 'C', 2: 'CC', 3: 'CCC', 
    4: 'B', 5: 'BB', 6: 'BBB', 
    7: 'A', 8: 'AA', 
    9: 'AAA (Tốt nhất)'
}

# Nếu số lượng bin tạo ra ít hơn 10 (do dữ liệu tập trung), code sẽ tự xử lý
df_results['Rating_Label'] = df_results['Rating_Class'].map(rating_map)
# Fill những hạng không có tên (nếu có lỗi map) bằng "Unrated"
df_results['Rating_Label'] = df_results['Rating_Label'].fillna('Unrated')

# 3. Tổng hợp bảng Master Scale
master_scale = df_results.groupby('Rating_Label').agg(
    Min_Score=('Score', 'min'),
    Max_Score=('Score', 'max'),
    Total_Count=('Target', 'count'),
    Bad_Count=('Target', 'sum')
).reset_index()

# Tính PD thực tế (Observed PD)
master_scale['Observed_PD'] = master_scale['Bad_Count'] / master_scale['Total_Count']

# Sắp xếp bảng theo Điểm tăng dần (Từ D lên AAA)
master_scale = master_scale.sort_values(by='Min_Score', ascending=True)

# Format lại số liệu cho đẹp
master_scale_display = master_scale.copy()
master_scale_display['Observed_PD'] = master_scale_display['Observed_PD'].apply(lambda x: f"{x:.2%}")

print("\n=== MASTER SCALE (BẢNG XẾP HẠNG TÍN DỤNG NỘI BỘ) ===")
display(master_scale_display[['Rating_Label', 'Min_Score', 'Max_Score', 'Total_Count', 'Observed_PD']])

# 4. Vẽ biểu đồ quan hệ giữa Hạng và Rủi ro (PD)
plt.figure(figsize=(12, 6))
# Vẽ cột số lượng khách hàng
ax1 = sns.barplot(data=master_scale, x='Rating_Label', y='Total_Count', color='lightblue', alpha=0.6)
ax1.set_ylabel('Số lượng Khách hàng', color='blue')
ax1.tick_params(axis='y', labelcolor='blue')

# Vẽ đường PD
ax2 = ax1.twinx()
sns.lineplot(data=master_scale, x='Rating_Label', y='Observed_PD', color='red', marker='o', linewidth=2, ax=ax2)
ax2.set_ylabel('Tỷ lệ nợ xấu (PD)', color='red')
ax2.tick_params(axis='y', labelcolor='red')

plt.title('Phân phối Khách hàng và Tỷ lệ Vỡ nợ theo Hạng tín dụng')
plt.grid(False)
plt.show()

In [ ]:
df.shape 

In [ ]:
df.columns